In [1]:
import pandas as pd
import numpy as np

import plotly.express as px
import cufflinks as cf
import chart_studio.plotly as py
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
#to make everything locally
cf.go_offline()

In [2]:
data = pd.read_csv("D:\data science\ineuron\Project\Business Analytics\Data_given\gpscleaned.csv")

In [3]:
data.head()

,App,Category,Rating,Reviews,Type,Content Rating,Genres,Size_New,Installs_New,Price_New,CurrentVer_New,AndroidVersion_New,LastUpdated_New
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,97,Free,Everyone,Art & Design,19.0,10000,0,100,40,2018-01-07
1,Coloring book moana,ART_AND_DESIGN,3.9,57,Free,Everyone,Art & Design;Pretend Play,14.0,500000,0,200,40,2018-01-15
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,42,Free,Everyone,Art & Design,8.7,5000000,0,124,40,2018-08-01
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,92,Free,Teen,Art & Design,25.0,50000000,0,Varies with device,42,2018-06-08
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,57,Free,Everyone,Art & Design;Creativity,2.8,100000,0,110,44,2018-06-20


In [4]:
data.shape

(10840, 13)

In [5]:
data.dtypes

App                    object
Category               object
Rating                float64
Reviews                 int64
Type                   object
Content Rating         object
Genres                 object
Size_New               object
Installs_New            int64
Price_New               int64
CurrentVer_New         object
AndroidVersion_New     object
LastUpdated_New        object
dtype: object

In [6]:
np.round(data.isnull().mean(),2)

App                   0.00
Category              0.00
Rating                0.14
Reviews               0.00
Type                  0.00
Content Rating        0.00
Genres                0.00
Size_New              0.00
Installs_New          0.00
Price_New             0.00
CurrentVer_New        0.00
AndroidVersion_New    0.00
LastUpdated_New       0.00
dtype: float64

# findings 1:
- Size_New, AndroidVersion_New : All the etries are numeric except 'Varies with Devices' so we will consider this as numerical columns
- LastUpdated_New: datetime column , we will treat this later
- 14% of the data is missing in Rating Feature 

# categorical Columns analysis

In [7]:
categorical_feature = [feature for feature in data.columns if data[feature].dtypes == 'object' and feature not in ['LastUpdated_New', 'Size_New','AndroidVersion_New']]
d = {}
for feature in categorical_feature:
    key = "total number of categories in " + feature
    d[key] = data[feature].nunique()
d

{'total number of categories in App': 9659,
 'total number of categories in Category': 33,
 'total number of categories in Type': 2,
 'total number of categories in Content Rating': 6,
 'total number of categories in Genres': 119,
 'total number of categories in CurrentVer_New': 1059}

In [8]:
data.Genres.value_counts()

Tools                                842
Entertainment                        623
Education                            549
Medical                              463
Business                             460
                                    ... 
Travel & Local;Action & Adventure      1
Racing;Pretend Play                    1
Role Playing;Education                 1
Trivia;Education                       1
Adventure;Brain Games                  1
Name: Genres, Length: 119, dtype: int64

# Findings2:
- App feature doesnot give much info
- 


# Which category has  highest number of installations

In [9]:
x_val = data['Installs_New'].groupby(by = data['Category']).sum().sort_values(ascending =False)


In [10]:
px.bar(data, x = x_val.index, y = x_val, color = x_val.index)

- Game has maximum number of installations followed by Communication, Productivity,Social, Tools

# Which Category has maximum average rating

In [11]:
x_rat_cat = data['Rating'].groupby(by = data['Category']).mean().sort_values(ascending =False)
px.bar(data, x=x_rat_cat.index, y = x_rat_cat.values, color = x_rat_cat)

- top 12 categories in terms rating include categories which has least number of installations except GAmes and Social, 
- categories with extremely low number of installations cant be compared with categories with extremely high installation in terms of ratings
- we will check the average ratings of top 10 categories n terms of installations 
- Game and Social categories are very much popular and favourable by the ppl

In [12]:
px.bar(data, x  = x_val.head(10).index, y = x_rat_cat[x_val.head(10).index], color = x_val.head(10).index)

- it seems like Game Category is performing good in terms of number of installations and rating  followed by social 
- Tools catgeory has the lowest rating among top installed categories, better apps in this category can be launched or the existing app can be looked upon for further improvements

# Which Category has maximum number of reviews submitted on GPS

In [13]:
# there are some negative signs before('-') we will remove them 
def positive(column):
    for i in column:
        if i>0:
            return i
        else:
            return i*(-1)
data['Reviews'] = data[['Reviews']].apply(positive,axis=1)

In [14]:
x_cat_rev = data['Reviews'].groupby(by = data['Category']).sum().sort_values(ascending =False)
px.bar(data, x = x_cat_rev.index, y = x_cat_rev.values, color = x_cat_rev.index)

- It seems like Family category apps have maximum reviews submitted, thtere could be many reasons for it one reason could be family is an integral part and ppl dont want to miss reporting any good/bad reviews on it so that others can consider those points
- Since Game category has maximum number of installation it was expected to have good amount of reveiws
- Events and last few categories have least number of reviews submitted which is obvious as users ar less in number for these categories

# Which of the highest Installed category has highest ratings submitted?

In [15]:
px.bar(data, x = x_cat_rev[x_val.head(10).index].index, y = x_cat_rev[x_val.head(10).index].values, color = x_cat_rev[x_val.head(10).index].index)

# Findings:
- Though the number of installation in Game category is highest but in terms of review collection it is very low.
- We can include a scheme wherein the user can have extra lives in the game if he submits review on the google play store or may be extension of trial. The reviews can then be used for improvements.
### The Benefits of Reviews for the Google Play Store as per https://www.makeuseof.com/why-you-should-leave-reviews-on-the-google-play-store/
- Leaving reviews for an app not only helps the users and the developers, but also the Google Play Store itself. The Play Store's goal as an app platform is to show you fast, accurate, and personalized results when you search for your desired app and keep spam apps away.

- To make that happen, it needs information on the app's performance—which is displayed via user reviews. An app that has a 4.5-star rating is most probably safer and more relevant than an app with a 2-star rating of the same genre.

- This information helps Google's algorithm better rank the apps on the Play Store and bring you high-quality app results that you are most likely to download and have a good experience with. The more favorable an app's rating and reviews, the more the number of people downloading that app and using Play Store's services.

# Which categories are of Type paid?

In [16]:
paid_data = data[data.Type=='Paid']
px.bar(paid_data,x = paid_data['Category'].value_counts().index, y = paid_data['Category'].value_counts().values, color = paid_data['Category'].value_counts().index)

# which are the categories with highest number of paid installations?

In [17]:
paid_cat_install = paid_data['Installs_New'].groupby(by = paid_data['Category']).sum().sort_values(ascending =False)
px.bar(paid_data, x = paid_cat_install.index, y = paid_cat_install.values , color =paid_cat_install)

# Findings
- Paid apps from Family category are installed maximum though in overall installations (paid+free) Game Category is at top with 35Billion + installations and Family at 6 Place with 10 Billion+ Installations
- There could be many reasons for this. One reason seems like app could be providing one to one consultation to the person which is more preferred over a generalised app which are mostly free

# Which category with paid apps has highest ratings?

In [18]:
px.bar(paid_data, x = paid_data['Rating'].groupby(by = paid_data['Category']).mean().sort_values().dropna().index, 
       y = paid_data['Rating'].groupby(by = paid_data['Category']).mean().sort_values().dropna().values,
       color = paid_data['Rating'].groupby(by = paid_data['Category']).mean().sort_values().dropna().index)

# Findings:
- The average rating of paid News And Magzines is highest howsoever Games which has highest number of installations and family with hihgest number of pai dinstallatons needs improvement
- Better Family apps can be of great scope 
- Paid parenting app has least average rating , This is the category which needs alot of improvement. Total Installations of Parenting Category Apps is 25K+ which is very less 

# Which category has highest number of Reviews submitted for paid apps?

In [19]:
paid_rev_cat = paid_data['Reviews'].groupby(by = paid_data['Category']).sum().sort_values(ascending = False)
px.bar(paid_data, x = paid_rev_cat.index, y = paid_rev_cat.values, color = paid_rev_cat.index, text = paid_cat_install)

# Which ContentRating has highest number of highest number of Installations?

In [20]:
install_cr = data['Installs_New'].groupby(by = data['Content Rating']).count().sort_values(ascending =False)

px.bar(data, x = install_cr.index, y = install_cr.values, color = install_cr.index)

In [21]:
data_fixedSize = data[data['Size_New']!='Varies with device']
data_fixedSize['Size_New'] = data_fixedSize['Size_New'].astype(float)
px.box(data_fixedSize, y = 'Size_New')

# Findings 
- Family apps and games categories have heavy apps which is very obvious